In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

#print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=False)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#Putting all the functions here
#1.1091571486954503

In [ ]:
# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev,diff_method='backprop')
def commutator_0(H,w, k):
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)
    #res = (qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev,diff_method='backprop')
def commutator_1(H,w, k):
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w)
    return qml.expval(res)

# Energy calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, diff_method='backprop')
def ash(params, ash_excitation, hf_state, qubits, H):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            #qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
            
    return qml.expval(H)

# Calculation of New state 
#print('Now we are going to calculate the new_state')
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, diff_method='backprop')
def new_state(hf_state, ash_excitation, fparams, params):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            print('The wires1 are', ash_excitation[i][2:][::-1])
            #qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            #qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
            

    return qml.state()

In [ ]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

print('The Hamiltonian is ', H)
adit = 6
fparams = []
excitations= []
operator_pool = (singles) + (doubles)
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)
        w = qml.fermi.jordan_wigner(i)
        if np.array_equal(k, hf_state):
            print('Print, if this is activated - HF state')
            current_value = abs(2*(commutator_0(H, w, k)))            #Removed the abs() function
        else:
            current_value = abs(2*(commutator_1(H, w, k)))           #Removed the abs() function
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    #ash_excitation.insert(0, excitations)  #Updated this statement
    ash_excitation.append(excitations)
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="backprop")
    print('Going to do energy calculation')
    for n in range(150):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', params)
    print('Updated excitation are', ash_excitation)
    
    # New state generation
    ostate = new_state(hf_state, ash_excitation, fparams, params)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,fparams,params))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


In [ ]:
print(qml.specs(cost_fn)(params, ash_excitation, hf_state, qubits, H))

## Implementing scipy Minimize

In [ ]:
from scipy.optimize import minimize



dev = qml.device('default.qubit', wires=8)
ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [2, 6]]

@qml.qnode(dev)
def circuit(ash_excitation, hf_state, params, H):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', [6,2])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
            
    return qml.expval(H)


def cost(params):
    energy = circuit(ash_excitation, hf_state, params, H)
    return energy

params = np.array([0.011, 0.012, 0.05, 0.5,1.0], requires_grad=True)

minimize(cost, params, method='BFGS')